In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
walks_nb=182183
generated_expts=1921314

In [ ]:
from rspace_loader import RSpaceLoader
loader =RSpaceLoader(url=os.getenv("RSPACE_URL"), api_key=os.getenv("RSPACE_API_KEY"), folder_id=walks_nb)
docs = loader.load()

In [ ]:
from langchain.chains.summarize import load_summarize_chain
import textwrap
chain = load_summarize_chain(llm=chat_llm, verbose=True, chain_type="map_reduce")
output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)


The above works very well, it does a good job at summarising. 3.5 turbo seems good, much faster than gpt4
TODO:
 - do this in streamlit
 - input field for a notebook or folder id. Support global Id or normal ID
 - Load docs and summarise. Note how many docs there are (max 10)
 - generate summary of each doc
 - generate summary of summaries of folder.

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.utils import filter_complex_metadata
from lan
db = Chroma.from_documents(docs, OpenAIEmbeddings())



In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdfLoader = PyPDFLoader('./data/inc.pdf')
splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=400)
docs = pdfLoader.load_and_split(text_splitter=splitter);

In [ ]:
chars = sum(len(x.page_content) for x in docs)
len(docs)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
chat_llm = ChatOpenAI(temperature=0.0)
prompt = """
    You are an expert in {research_area} research and will be given documents to summarise.
    Please summarise the following text.
    Try very hard to extract the key points.
    Try very hard to not  repeat the text verbatim.
    Your response must be less than one third of the length of the input text. Use bullet points and terse sentences.

    Extract a maximum of 10 keywords and list these at the end in a section called 'Keywords'.
    
    The text:
    
    {text}
    
    Your summary:
    
    Your keywords (maximum 10):
    """
promptT = PromptTemplate(input_variables=['text', 'research_area'],template=prompt)
llm_chain = LLMChain(llm=chat_llm, prompt=promptT)

In [ ]:
with open('polosummary-gpt4output.txt', 'r') as input:
    lines = input.readlines()

In [ ]:
from langchain.docstore.base import Document
content  = [''.join(lines)]
summarisedOnce = [Document(page_content=s1, metadata={'source':f"page-{i}"}) for i, s1 in enumerate(content)]
summarisedAndSplit= splitter.split_documents(summarisedOnce)
len(content[0])

In [ ]:
from langchain.chains.llm import LLMChain
outputs2 = []
for d in docs:
    resp = llm_chain.predict(text=d.page_content, research_area='biomedical')
    outputs2.append(resp)
    

In [ ]:
t = ''.join(outputs2)
d2 = Document(page_content=t, metadata=)

In [ ]:
d2_split =splitter.split_documents([d2])

In [ ]:
d2_split

In [ ]:
from langchain.chains.llm import LLMChain
outputs23 = []
for d in d2_split:
    resp = llm_chain.predict(text=d.page_content, research_area='biomedical')
    outputs23.append(resp)

In [ ]:
from langchain.chains.summarize import load_summarize_chain
import textwrap
chain = load_summarize_chain(llm=chat_llm, verbose=True, chain_type="map_reduce")
output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)